In [13]:
import requests
from dotenv import load_dotenv
import os
import mysql.connector
import json
import csv
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import constants
import driver
import importlib

In [14]:
importlib.reload(driver)
importlib.reload(constants)
from driver import Driver
from constants import DB_DRIVER_NAMES, YEARS, TYRE_PARAMS, DEFAULT_BASE_LAP, DRIVER_TYRE_CONSTANTS

In [15]:
load_dotenv()

db_host = os.getenv('DB_HOST')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_name = os.getenv('DB_NAME')

In [16]:
quali_df = pd.read_csv('all_quali_data.csv')
race_df = pd.read_csv('all_race_data.csv')

In [17]:
#This method returns the average pitstop loss for a given race. At the moment it is just the Dutch GP
#More will be added later
def get_avg_pit_loss():
    return float(21.355)

In [18]:
def get_base_time(race, year, driver):
    base_time = (quali_df[(quali_df.year == year) & (quali_df.race_name == f"{race}") & (quali_df.driver == f"{driver}")])
    if base_time.empty:
        return DEFAULT_BASE_LAP[f"{race}"]
    return float(base_time.iloc[0, 3])

In [19]:
#This method is to check weather a pit stop is now a smart option
#For the basic implementation we will just assume that if the tyre has started to drop off then a pit stop is needed
def time_to_pit(tyre_compound, num_laps, stint_lap, lap_number, lap_time, lap_times):

    remaining_laps = num_laps - lap_number

    #print(f"remaining laps: {remaining_laps} ")
    if remaining_laps < 0:
        return True
    elif remaining_laps < 10 or stint_lap < 6:

        return False

    elif tyre_compound == "soft":

        for i in range(1, 3):

            if(lap_times[i]+ 1 < lap_time):

                return True
            
        return False
    
    elif tyre_compound == "medium":

        for i in range(1, 3):

            if(lap_times[i]+ 1.9 < lap_time):

                return True
            
        return False
    
    else:

        for i in range(1, 3):

            if(lap_times[i]+ 2.3 < lap_time):

                return True
            
        return False

In [20]:
def simulate_pit_stop(current_compound, num_laps, lap_number):
    laps_remaining = num_laps - lap_number
    if current_compound == "soft":
        if laps_remaining > 25:
            next_compound = "hard"
        else:
            next_compound = "medium"
    elif current_compound == "medium":
        if laps_remaining < 13:
            next_compound = "soft"
        else:
            next_compound = "hard"
    else:
        if laps_remaining < 13:
            next_compound = "soft"
        else:
            next_compound = "medium"
    return next_compound

In [21]:
drivers = []

In [22]:
#Need to change the starting compound variable so it is changeable - predictable?
def simulate_race(race, year, starting_compound, num_laps):
    for i in range(len(DB_DRIVER_NAMES)):
        drivers.append(Driver(driver_id=DB_DRIVER_NAMES[i],
                              base_lap_time=get_base_time(race, year, DB_DRIVER_NAMES[i]),
                              fuel_level = 110,
                              compound=starting_compound))
    lap = 1
    while lap <= num_laps:
        print(f"simulating lap {lap}")
        for driver in drivers:
            if driver.current_lap < num_laps:
                driver.stint_lap += 1
                lap_time, sector_times = driver.simulate_lap(driver.stint_lap, race)
                print(f"Driver {driver.driver_id} has lap time {lap_time}")
                print(f"Driver {driver.driver_id} has sector times {sector_times} for lap {lap}")
                for sector_time in sector_times:
                    driver.race_time += sector_time
                driver.race_time += lap_time
                if time_to_pit(driver.compound, num_laps, driver.stint_lap, lap, lap_time, driver.lap_times):
                    driver.compound = simulate_pit_stop(driver.compound, num_laps, lap)
                    driver.race_time += get_avg_pit_loss() #Add on the time loss due to pitting
                    driver.stint_lap = 1
        drivers.sort(key=lambda d: d.race_time)
        for idx, driver in enumerate(drivers):
            driver.position = idx + 1
        for driver in drivers:
            print(f"Driver {driver.driver_id} is in position {driver.position}")
        #print(f"Intervals after lap {lap}")
        for i in range(1, len(drivers)):
            interval = drivers[i].race_time - drivers[i-1].race_time
            #print(f"Driver {drivers[i].driver_id} is {interval:.3f} seconds behind Driver {drivers[i-1].driver_id}")
        lap += 1
        print("\n")
    drivers.sort(key=lambda d: d.race_time)
    for idx, driver in enumerate(drivers):
        driver.position = idx + 1
    return drivers
                    
    

In [23]:
drivers = simulate_race("netherlands", 2024, "medium", 71)
for driver in drivers:
    driver_name = driver.driver_id
    total_race_time = driver.race_time
    print(f"Driver: {driver_name} had race time of: {total_race_time} in position {driver.position} base lap time: {driver.base_lap_time}")

simulating lap 1
Driver carlos-sainz-jr has lap time 78.309361
Driver carlos-sainz-jr has sector times [3.3712179910499995, 15.50916894605, 9.833306460769998, 21.01588321157, 6.249087007799999, 13.508364772499998, 8.822332610259998] for lap 1
Driver alexander-albon has lap time 80.7196555
Driver alexander-albon has sector times [3.474981169275, 15.986527771775, 10.135967141135, 21.662733946535, 6.4414285089, 13.92414057375, 9.09387638863] for lap 1
Driver charles-leclerc has lap time 77.96932299999999
Driver charles-leclerc has sector times [3.356579355149999, 15.441824420149999, 9.790607889109998, 20.924627213509996, 6.221951975399999, 13.449708217499998, 8.784023929179998] for lap 1
Driver daniel-ricciardo has lap time 79.4488835
Driver daniel-ricciardo has sector times [3.4202744346749996, 15.734851377175, 9.976396301094999, 21.321696864895, 6.340020903299999, 13.704932403749998, 8.95071121511] for lap 1
Driver fernando-alonso has lap time 78.06453199999999
Driver fernando-alonso ha